In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from tcn import TemporalConvNet
import time
import os


In [2]:
from tcnae import encoder,decoder

In [3]:
import data

In [4]:
data_dir = os.environ['MYDATA_DIR']

In [5]:
data_c = data.Data(wd=data_dir)

In [6]:
raw_x, raw_y = data_c.data(class_num=19, data_num=400)

In [7]:
raw_y.shape

(7600,)

In [8]:
raw_x.shape

(7600, 125, 3)

In [9]:
batch_size = 50
seq_len = 125
epochs = 50
iters = 100
T = 130
n_steps = T + (2 * seq_len)
n_classes = 10  # Digits 0 - 9
n_train = 10000
n_test = 1000

In [10]:
X_train, X_test, y_train, y_test = train_test_split(raw_x, raw_y, test_size=0.90, random_state=0)
X_train.shape

(760, 125, 3)

In [11]:
X_train_new = X_train
y_train_new = y_train
for _ in range(3):
    for i,j in zip(X_train,y_train):
        X_train_new = np.append(X_train_new,i+np.random.normal(0, 1, (125, 3)).reshape(1,125,3))
        y_train_new = np.append(y_train_new, j)
X_train = X_train_new.reshape(-1,125,3)
y_train = y_train_new

In [12]:
num_chans = [6] * (3) 
encoder = encoder(num_inputs=raw_x.shape[1],num_outputs=19, num_channels=num_chans, kernel_size=4, dropout=0.0).cuda()

In [13]:
decoder = decoder(num_inputs=19,num_outputs=3, num_channels=num_chans, kernel_size=4, dropout=0.0).cuda()

In [14]:
criterion = nn.MSELoss().cuda()
lr = 0.001
optimizer1 =optim.Adam(encoder.parameters(),lr=lr)
optimizer2 =optim.Adam(decoder.parameters(),lr=lr)

In [15]:
encoder.train()
decoder.train()
total_loss = 0
start_time = time.time()
correct = 0
counter = 0

In [16]:
optimizer1

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [34]:
input = torch.tensor(X_train,dtype=torch.float32).cuda()
for i in range(1):
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    z = encoder(input)
    out = decoder(z)
    loss = criterion(out, input)
    if i%50==0:
#         input = torch.tensor([X_train[1]],dtype=torch.float32)
#         out= model(input)
#         print(out)
#         print(torch.max(out, 1))
        print(i,loss)
    loss.backward()
    optimizer1.step()
    optimizer2.step()

0 tensor(4.4951, device='cuda:0', grad_fn=<MseLossBackward>)


/mnt/nfs/mizuno/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [35]:
input = torch.tensor([X_train[10]],dtype=torch.float32).cuda()


In [36]:
z = encoder(input)
z

tensor([[[-1.8294,  1.1112, -0.9947,  0.7173,  1.1819,  0.5137,  0.5710,
          -1.6919,  0.6121, -0.1167, -0.9403,  1.4942,  2.0087,  0.9213,
          -1.6277,  0.2236, -0.8367, -0.6466, -1.2254],
         [-1.4565, -0.0002, -1.9600,  0.5660, -1.2956,  0.3850, -1.3586,
          -0.3687, -1.4045,  2.8192,  0.0360,  0.6725,  2.2526,  0.2706,
           0.9487, -0.0834, -0.4490, -3.4515, -1.5058],
         [-0.7110,  1.5395,  2.0737, -1.6701,  2.9501,  2.3050,  0.7409,
          -2.1365,  0.2616,  5.8701,  0.0673,  2.0059,  2.9395, -1.9414,
           1.3012, -2.1805,  2.7000,  1.2605, -3.1883],
         [ 0.2005, -1.4058, -0.5086,  0.1271, -2.0967, -2.3348,  2.1216,
          -0.0943, -1.0376,  3.4061, -0.2950, -1.0806,  0.6493, -1.2225,
          -0.8572,  1.2820, -0.8509, -4.1005,  1.4753],
         [ 0.8127, -2.0373, -4.7150,  7.9344, -1.8534,  1.3460,  1.8379,
          -2.8648,  2.4520, -1.2657,  2.0379, -2.7865,  0.3182, -2.1675,
           1.2777,  3.9285, -5.5287,  2.6578, 

In [33]:
z

tensor([[[-1.8761,  1.1610, -0.9509,  0.7540,  1.2114,  0.4914,  0.5937,
          -1.7213,  0.6376, -0.1382, -0.9423,  1.4700,  2.0349,  0.9585,
          -1.6994,  0.2769, -0.8362, -0.5859, -1.2205],
         [-1.4677,  0.0333, -1.9388,  0.5728, -1.2533,  0.4133, -1.3878,
          -0.4302, -1.4341,  2.7875,  0.0458,  0.6602,  2.2504,  0.2575,
           0.9218, -0.0119, -0.4029, -3.4952, -1.5049],
         [-0.6476,  1.5663,  2.0928, -1.6783,  2.9295,  2.3833,  0.6946,
          -2.1924,  0.2160,  5.8589,  0.0694,  2.0158,  2.9439, -1.9226,
           1.3034, -2.2152,  2.6977,  1.1871, -3.0955],
         [ 0.1228, -1.3930, -0.5531,  0.0427, -2.1845, -2.3864,  2.1392,
          -0.1016, -1.0690,  3.4628, -0.2816, -1.1104,  0.6565, -1.2180,
          -0.8783,  1.2725, -0.8750, -4.0879,  1.5148],
         [ 0.7378, -2.0334, -4.7232,  7.9609, -1.8598,  1.4046,  1.8626,
          -2.8586,  2.4929, -1.3778,  2.0451, -2.8462,  0.2540, -2.1311,
           1.1861,  3.9507, -5.6387,  2.7299, 